In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras import backend as K

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense

In [8]:
from sklearn import model_selection

In [9]:
import math

In [10]:
import tensorflow as tf

In [11]:
import horovod.keras as hvd

In [12]:
from liz_models import distance_generation

In [13]:
from liz_models import weights_generation

In [14]:
df = pd.DataFrame()

In [15]:
df['distances'] = distance_generation()

In [16]:
df['weights'] = weights_generation()

In [17]:
df[['distances','weights']].count()

distances    100000
weights      100000
dtype: int64

In [18]:
df['prices'] = np.array([((df['distances'][i] * df['weights'][i])/100) for i in range(len(df['distances']))])

In [19]:
df.head()

,distances,weights,prices
0,299,475,1420.25
1,585,193,1129.05
2,242,224,542.08
3,881,469,4131.89
4,410,444,1820.40


In [20]:
hvd.init()

0

In [21]:
config = tf.ConfigProto()

In [22]:
config.gpu_options.allow_growth = True

In [23]:
config.gpu_options.visible_device_list = str(hvd.local_rank())

In [24]:
K.set_session(tf.Session(config=config))

In [25]:
batch_size = 128

In [26]:
num_classes = 10

In [27]:
epochs = int(math.ceil(120.0) / hvd.size())

In [28]:
x, x_val, y, y_val = model_selection.train_test_split(df['prices'], df['distances'], test_size=0.2, random_state=42)

In [29]:
dlce = lambda l: [v for v in l if v >= 0]

In [30]:
x = np.array(dlce(x))
x_val = np.array(dlce(x_val))

In [31]:
y = np.array(dlce(y))
y_val = np.array(dlce(y_val))

In [32]:
opt = keras.optimizers.RMSprop(1.0 * hvd.size())

In [33]:
opt = hvd.DistributedOptimizer(opt)

In [34]:
model = Sequential()

In [35]:
model.add(Dense(1, input_dim=1))

In [36]:
model.compile(optimizer=opt, loss=keras.losses.mse,metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [37]:
callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]

In [38]:
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoints/checkpoint-{epoch}.h5'))

In [39]:
model.fit(x, y, batch_size=batch_size, callbacks=callbacks, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

Train on 80000 samples, validate on 20000 samples
Epoch 1/120
80000/80000 [==============================] - 1s - loss: 810630.7076 - acc: 0.0014 - val_loss: 135188.6257 - val_acc: 0.0015
Epoch 2/120
80000/80000 [==============================] - 1s - loss: 802815.5433 - acc: 8.3750e-04 - val_loss: 775234.4793 - val_acc: 5.5000e-04
Epoch 3/120
80000/80000 [==============================] - 1s - loss: 801816.0925 - acc: 7.3750e-04 - val_loss: 194504.6890 - val_acc: 8.0000e-04
Epoch 4/120
80000/80000 [==============================] - 1s - loss: 791050.2444 - acc: 7.2500e-04 - val_loss: 2807161.4684 - val_acc: 4.0000e-04
Epoch 5/120
80000/80000 [==============================] - 1s - loss: 793347.2640 - acc: 7.2500e-04 - val_loss: 2831228.7292 - val_acc: 0.0000e+00
Epoch 6/120
80000/80000 [==============================] - 1s - loss: 798796.8859 - acc: 5.2500e-04 - val_loss: 509777.0514 - val_acc: 8.5000e-04
Epoch 7/120
80000/80000 [==============================] - 1s - loss: 790985.595

80000/80000 [==============================] - 1s - loss: 794781.6982 - acc: 6.1250e-04 - val_loss: 392427.2121 - val_acc: 5.5000e-04
Epoch 57/120
80000/80000 [==============================] - 1s - loss: 795069.9380 - acc: 5.7500e-04 - val_loss: 84582.2644 - val_acc: 0.0015
Epoch 58/120
80000/80000 [==============================] - 1s - loss: 788999.2955 - acc: 6.0000e-04 - val_loss: 535763.3577 - val_acc: 3.5000e-04
Epoch 59/120
80000/80000 [==============================] - 1s - loss: 798675.5667 - acc: 5.6250e-04 - val_loss: 276435.5221 - val_acc: 8.5000e-04
Epoch 60/120
80000/80000 [==============================] - 1s - loss: 794632.9675 - acc: 7.0000e-04 - val_loss: 1750524.5978 - val_acc: 5.0000e-05
Epoch 61/120
80000/80000 [==============================] - 1s - loss: 798666.5045 - acc: 4.8750e-04 - val_loss: 88943.4232 - val_acc: 0.0013
Epoch 62/120
80000/80000 [==============================] - 1s - loss: 779165.6916 - acc: 7.8750e-04 - val_loss: 1262522.1158 - val_acc: 2.0

80000/80000 [==============================] - 1s - loss: 795321.4996 - acc: 7.3750e-04 - val_loss: 644500.3505 - val_acc: 1.5000e-04
Epoch 112/120
80000/80000 [==============================] - 1s - loss: 792540.3880 - acc: 6.6250e-04 - val_loss: 1915728.9752 - val_acc: 1.5000e-04
Epoch 113/120
80000/80000 [==============================] - 1s - loss: 797981.1635 - acc: 6.5000e-04 - val_loss: 40015.0258 - val_acc: 0.0014
Epoch 114/120
80000/80000 [==============================] - 1s - loss: 783551.1344 - acc: 6.2500e-04 - val_loss: 725330.1676 - val_acc: 4.5000e-04
Epoch 115/120
80000/80000 [==============================] - 1s - loss: 798869.1260 - acc: 3.6250e-04 - val_loss: 950720.6604 - val_acc: 1.5000e-04
Epoch 116/120
80000/80000 [==============================] - 1s - loss: 786636.2734 - acc: 6.6250e-04 - val_loss: 291122.7661 - val_acc: 5.0000e-04
Epoch 117/120
80000/80000 [==============================] - 1s - loss: 799767.9628 - acc: 5.1250e-04 - val_loss: 717118.1183 - va

In [40]:
score = model.evaluate(x_val, y_val, verbose=0)

In [41]:
print('Test loss:', score[0])

Test loss: 278487.9772875


In [42]:
print('Test accuracy:', score[1])

Test accuracy: 0.00055


In [43]:
print(model.predict(np.array([np.random.randint(10,100) for _ in range(5)])))

[[265.8249 ]
 [232.38947]
 [259.7029 ]
 [247.92987]
 [239.92421]]


In [44]:
model.save_weights('travel-price.hdf5')

In [45]:
with open('travel-price.json','w') as f:
    f.write(model.to_json())
    f.close()